## Скрипт для сортировки fMRI данных по возрастным когортам и конвертации во flat BIDS-совместимую структуру

Данный скрипт предназначен для сортировки fMRI данных по возрастным когортам.
Он читает информацию о возрасте участников из файлов `cohort_teen_participants.сsv` и `cohort_adult_participants.сsv`, а затем перемещает соответствующие файлы fMRI в папки, соответствующие их возрастной группе.

In [79]:
# %pip install --upgrade pandas

In [80]:
import pandas as pd
import numpy as np
import os, shutil, re
import subprocess, shlex
from pathlib import Path

# Настраиваем отображение графиков в Jupyter Notebook
%matplotlib inline

# Настраиваем отображение всех колонок DataFrame
pd.set_option('display.max_columns', 100)  # Ограничение по колонкам для удобства
pd.set_option('display.max_rows', 1000)  # Ограничение по строкам для удобства
pd.set_option('display.width', 1000)  # Ширина отображения

In [81]:
TEEN_MINIMAL_AGE = 12
TEEN_MAXIMAL_AGE = 15.99
ADULT_MINIMAL_AGE = 16
ADULT_MAXIMAL_AGE = 21.99

### 1. Загрузка данных

In [82]:
print("\n--- 1. Загрузка данных ---")

# Укажите правильный путь к вашему файлу
file_path_cohort_teen_participants = 'cohort_teen_participants.csv'
file_path_cohort_adult_participants = 'cohort_adult_participants.csv'

# Загружаем .csv файлы в DataFrame с обработкой ошибки
try:
    df_teen = pd.read_csv(file_path_cohort_teen_participants)
    print(f"Файл '{file_path_cohort_teen_participants}' успешно загружен.")
except FileNotFoundError:
    print(f"Ошибка: Файл '{file_path_cohort_teen_participants}' не найден. Пожалуйста, убедитесь, что он находится в правильной директории.")

try:
    df_adult = pd.read_csv(file_path_cohort_adult_participants)
    print(f"Файл '{file_path_cohort_adult_participants}' успешно загружен.")
except FileNotFoundError:
    print(f"Ошибка: Файл '{file_path_cohort_adult_participants}' не найден. Пожалуйста, убедитесь, что он находится в правильной директории.")


--- 1. Загрузка данных ---
Файл 'cohort_teen_participants.csv' успешно загружен.
Файл 'cohort_adult_participants.csv' успешно загружен.


### 2. Первичный осмотр данных

In [83]:
print("\n--- 2. Первичный осмотр данных ---")
    
# Вывод первых 5 строк для общего представления
print(f"\nПервые 5 строк данных для возрастной когорты {TEEN_MINIMAL_AGE}-{TEEN_MAXIMAL_AGE} лет:")
print(df_teen.head())
print(f"\nПервые 5 строк данных для возрастной когорты {ADULT_MINIMAL_AGE}-{ADULT_MAXIMAL_AGE} год:")
print(df_adult.head())


--- 2. Первичный осмотр данных ---

Первые 5 строк данных для возрастной когорты 12-15.99 лет:
   ScanDir ID  Site Gender    Age  Handedness  DX   Secondary Dx  ADHD Measure  ADHD Index  Inattentive  Hyper/Impulsive  IQ Measure  Verbal IQ  Performance IQ  Full2 IQ  Full4 IQ  Med Status  QC_Athena  QC_NIAK
0     2026113     3      F  12.99         1.0   1            NaN           2.0        90.0         89.0             78.0         1.0      122.0           108.0       NaN     106.0         1.0        1.0      1.0
1     1623716     3      F  12.65         1.0   1            NaN           2.0        87.0         90.0             90.0         1.0       89.0            88.0       NaN      89.0         1.0        1.0      1.0
2     1594156     3      M  12.87         1.0   0  Simple Phobia           2.0        54.0         55.0             57.0         1.0      124.0           104.0       NaN     108.0         1.0        1.0      1.0
3     2917777     3      M  12.66         1.0   0       

### 3. Сортировка данных по возрастным когортам

In [84]:
print("\n--- 3. Сортировка данных по возрастным когортам ---")

# Установим таблицу соответствия для источника данных (столбец 'Site')
site_mapping = {
    1: ['Peking_1', 'Peking_2', 'Peking_3'],
    2: ['Brown'],
    3: ['KKI'],
    4: ['NeuroIMAGE'],
    5: ['NYU'],
    6: ['OHSU'],
    7: ['Pittsburgh'],
    8: ['WashU'],
}

# Функция для получения списка источников по коду
def get_sites(code):
    return site_mapping.get(code, [])
# Добавляем колонку с источниками данных
df_teen['Sites'] = df_teen['Site'].apply(get_sites)
df_adult['Sites'] = df_adult['Site'].apply(get_sites)
print("\nКолонка 'Sites' с источниками данных добавлена.")

# Переименуем 'ScanDir ID' в 'ScanDirID' для удобства
df_teen.rename(columns={'ScanDir ID': 'ScanDirID'}, inplace=True)
df_adult.rename(columns={'ScanDir ID': 'ScanDirID'}, inplace=True)
print("\nКолонка 'ScanDir ID' переименована в 'ScanDirID'.")

# Преобразуем колонку 'ScanDir ID' в строковый тип
df_teen['ScanDirID'] = df_teen['ScanDirID'].astype(str)
df_adult['ScanDirID'] = df_adult['ScanDirID'].astype(str)
print("\nКолонка 'ScanDir ID' преобразована в строковый тип.")

# Если в колонке 'ScanDir ID' значение менее 5 символов или меньше, добавляем 2 ведущих нуля
df_teen['ScanDirID'] = df_teen['ScanDirID'].apply(lambda x: x.zfill(7) if len(x) == 5 else x)
df_teen['ScanDirID'] = df_teen['ScanDirID'].apply(lambda x: x.zfill(8) if len(x) == 6 else x)
df_adult['ScanDirID'] = df_adult['ScanDirID'].apply(lambda x: x.zfill(7) if len(x) == 5 else x)
df_adult['ScanDirID'] = df_adult['ScanDirID'].apply(lambda x: x.zfill(8) if len(x) == 6 else x)
print("\nКолонка 'ScanDirID' отформатирована с ведущими нулями.")

# Выводим количество уникальных источников данных
unique_sites_teen = set(site for sites in df_teen['Sites'] for site in sites)
unique_sites_adult = set(site for sites in df_adult['Sites'] for site in sites)
print(f"\nУникальные источники данных для возрастной когорты {TEEN_MINIMAL_AGE}-{TEEN_MAXIMAL_AGE} лет: {unique_sites_teen}")
print(f"Уникальные источники данных для возрастной когорты {ADULT_MINIMAL_AGE}-{ADULT_MAXIMAL_AGE} год: {unique_sites_adult}")

# Выводим первые 5 строк с новой колонкой
print(f"\nПервые 5 строк данных для возрастной когорты {TEEN_MINIMAL_AGE}-{TEEN_MAXIMAL_AGE} лет с новой колонкой 'Sites':")
print(df_teen.head())
print(f"\nПервые 5 строк данных для возрастной когорты {ADULT_MINIMAL_AGE}-{ADULT_MAXIMAL_AGE} год с новой колонкой 'Sites':")
print(df_adult.head())


--- 3. Сортировка данных по возрастным когортам ---

Колонка 'Sites' с источниками данных добавлена.

Колонка 'ScanDir ID' переименована в 'ScanDirID'.

Колонка 'ScanDir ID' преобразована в строковый тип.

Колонка 'ScanDirID' отформатирована с ведущими нулями.

Уникальные источники данных для возрастной когорты 12-15.99 лет: {'Peking_1', 'Peking_2', 'NeuroIMAGE', 'Peking_3', 'WashU', 'NYU', 'KKI', 'Pittsburgh', 'OHSU'}
Уникальные источники данных для возрастной когорты 16-21.99 год: {'Peking_2', 'Peking_1', 'NeuroIMAGE', 'Peking_3', 'WashU', 'NYU', 'Pittsburgh'}

Первые 5 строк данных для возрастной когорты 12-15.99 лет с новой колонкой 'Sites':
  ScanDirID  Site Gender    Age  Handedness  DX   Secondary Dx  ADHD Measure  ADHD Index  Inattentive  Hyper/Impulsive  IQ Measure  Verbal IQ  Performance IQ  Full2 IQ  Full4 IQ  Med Status  QC_Athena  QC_NIAK  Sites
0   2026113     3      F  12.99         1.0   1            NaN           2.0        90.0         89.0             78.0         1

### 4. Копирование файлов в соответствующие директории

In [85]:
# === Параметры ===

# Путь к корневой директории с исходными данными. В этой директории должны быть папки с данными субъектов, разделённые по источникам (например, 'Peking_1', 'Brown' и т.д.)
SOURCE_ROOT = "../../Materials/ADHD200/RawDataBIDS"

# Путь к директории, куда будут скопированы данные для возрастной когорты {TEEN_MINIMAL_AGE}-{TEEN_MAXIMAL_AGE} лет
DEST_ROOT_TEEN = "SortedRawDataBIDS/cohort_teen_participants"

# Путь к директории, куда будут скопированы данные для возрастной когорты {ADULT_MINIMAL_AGE}-{ADULT_MAXIMAL_AGE} год
DEST_ROOT_ADULT = "SortedRawDataBIDS/cohort_adult_participants"

# Путь к CSV файлам с информацией о субъектах
PARTICIPANTS_CSV_TEEN = DEST_ROOT_TEEN + "/participants_teen.csv"      # CSV с колонками ниже
PARTICIPANTS_CSV_ADULT = DEST_ROOT_ADULT + "/participants_adult.csv"   # CSV с колонками ниже

# Имена колонок в participants.csv
ID_COLUMN    = "ScanDirID"
SITE_COLUMN  = "Site"
SITES_COLUMN = "Sites"          # список каталога сайтов, например "[Peking_1,Peking_2,Peking_3]"
AGE_COLUMN   = "Age"
SEX_COLUMN   = "Gender"
DIAGNOSIS_COLUMN = "DX"

# Режим переноса: 'symlink' (быстро, экономит место) или 'copy'
MODE = "symlink"

In [86]:
print("\n--- 4. Копирование данных в новые директории ---")

# Проверим, что SOURCE_ROOT существует
if not os.path.exists(SOURCE_ROOT):
    raise FileNotFoundError(f"Исходная директория '{SOURCE_ROOT}' не найдена.")
else:
    print(f"\nИсходная директория '{SOURCE_ROOT}' найдена. Продолжаем...")

# Создадим новые специальные df только с необходимыми колонками ('ScanDir ID', 'Age', 'DX', 'Site', 'Sites')
df_teen_ = df_teen[['ScanDirID', 'Age','Gender', 'DX', 'Site', 'Sites']].copy()
df_adult_ = df_adult[['ScanDirID', 'Age', 'Gender', 'DX', 'Site', 'Sites']].copy()
print("\nСозданы новые DataFrame с необходимыми колонками ('ScanDirID', 'Age', 'DX', 'Site', 'Sites').")

# Удалим DEST_ROOT_TEEN и DEST_ROOT_ADULT, если они существуют, чтобы избежать конфликтов
if os.path.exists(DEST_ROOT_TEEN):
    shutil.rmtree(DEST_ROOT_TEEN)
    print(f"\nУдалена существующая директория {DEST_ROOT_TEEN} для предотвращения конфликтов.")
if os.path.exists(DEST_ROOT_ADULT):
    shutil.rmtree(DEST_ROOT_ADULT)
    print(f"\nУдалена существующая директория {DEST_ROOT_ADULT} для предотвращения конфликтов.")

# Создадим директории назначения, если они не существуют
os.makedirs(DEST_ROOT_TEEN, exist_ok=True)
os.makedirs(DEST_ROOT_ADULT, exist_ok=True)
print(f"\nСозданы директории назначения: {DEST_ROOT_TEEN} и {DEST_ROOT_ADULT}.")

# Сохраним новые DataFrame в .csv файлы для дальнейшего использования
df_teen_.to_csv(PARTICIPANTS_CSV_TEEN, index=False)
df_adult_.to_csv(PARTICIPANTS_CSV_ADULT, index=False)
print("\nНовые DataFrame сохранены в .csv файлы.")


--- 4. Копирование данных в новые директории ---

Исходная директория '../../Materials/ADHD200/RawDataBIDS' найдена. Продолжаем...

Созданы новые DataFrame с необходимыми колонками ('ScanDirID', 'Age', 'DX', 'Site', 'Sites').

Созданы директории назначения: SortedRawDataBIDS/cohort_teen_participants и SortedRawDataBIDS/cohort_adult_participants.

Новые DataFrame сохранены в .csv файлы.


Мини-обёртка для скрипта `flatten_bids_from_participants.py`.

In [87]:
# Небольшая обёртка над flatten_bids_from_participants.py

def run_command(command):
    """Выполняет команду в оболочке и возвращает вывод."""
    print(f"Выполняется команда: {command}")
    args = shlex.split(command)
    result = subprocess.run(args, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"Ошибка при выполнении команды: {result.stderr}")
    return result.stdout

def flatten_bids(participants_csv, source_root, dest_root, id_column, site_column, sites_column, sex_column, age_column, diagnosis_column, mode="symlink", dry_run=True):
    """Запускает flatten_bids_from_participants.py с заданными параметрами."""
    conv = Path("flatten_bids_from_participants.py")
    if not conv.exists():
        raise FileNotFoundError("Не найден flatten_bids_from_participants.py.")

    cmd = f"python {conv} --participants-csv {participants_csv} --source-root {source_root} --dest-root {dest_root} "\
          f"--id-column {id_column} --site-column {site_column} --sites-column {sites_column} "\
          f"--sex-column {sex_column} --age-column {age_column} --diagnosis-column {diagnosis_column} "\
          f"--mode {mode}"
    if dry_run:
        cmd += " --dry-run"
    output = run_command(cmd)
    return output

### 4.1 DRY-RUN (проверка без записи)
Скрипт пройдётся по CSV и сгенерирует **отчёт** о предполагаемых действиях в `flatten_report.tsv`,
но **ничего не будет записано** в `DEST_ROOT`.

In [88]:
print(f"\n--- 4.1.1 DRY-RUN (проверка без записи) на возрастной когорте {TEEN_MINIMAL_AGE} - {TEEN_MAXIMAL_AGE} лет ---")

ret = flatten_bids(PARTICIPANTS_CSV_TEEN, SOURCE_ROOT, DEST_ROOT_TEEN, ID_COLUMN, SITE_COLUMN, SITES_COLUMN, SEX_COLUMN, AGE_COLUMN, DIAGNOSIS_COLUMN, mode=MODE, dry_run=True)
print("\nКод возврата:", ret)

report = Path(DEST_ROOT_TEEN) / "flatten_report.tsv"
print("Ожидаемый отчёт:", report)


--- 4.1.1 DRY-RUN (проверка без записи) на возрастной когорте 12 - 15.99 лет ---
Выполняется команда: python flatten_bids_from_participants.py --participants-csv SortedRawDataBIDS/cohort_teen_participants/participants_teen.csv --source-root ../../Materials/ADHD200/RawDataBIDS --dest-root SortedRawDataBIDS/cohort_teen_participants --id-column ScanDirID --site-column Site --sites-column Sites --sex-column Gender --age-column Age --diagnosis-column DX --mode symlink --dry-run

Код возврата: Done. See: /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/ADHD_connectome_biomarkers_transfer_research/ADHD200/SortedRawDataBIDS/cohort_teen_participants/flatten_report.tsv and /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/ADHD_connectome_biomarkers_transfer_research/ADHD200/SortedRawDataBIDS/cohort_teen_participants/participants.tsv

Ожидаемый отчёт: SortedRawDataBIDS/cohort_teen_participants/flatten_report.tsv


In [89]:
print(f"\n--- 4.1.2 DRY-RUN (проверка без записи) на возрастной когорте {ADULT_MINIMAL_AGE} - {ADULT_MAXIMAL_AGE} лет ---")

ret = flatten_bids(PARTICIPANTS_CSV_ADULT, SOURCE_ROOT, DEST_ROOT_ADULT, ID_COLUMN, SITE_COLUMN, SITES_COLUMN, SEX_COLUMN, AGE_COLUMN, DIAGNOSIS_COLUMN, mode=MODE, dry_run=True)
print("\nКод возврата:", ret)

report = Path(DEST_ROOT_ADULT) / "flatten_report.tsv"
print("Ожидаемый отчёт:", report)


--- 4.1.2 DRY-RUN (проверка без записи) на возрастной когорте 16 - 21.99 лет ---
Выполняется команда: python flatten_bids_from_participants.py --participants-csv SortedRawDataBIDS/cohort_adult_participants/participants_adult.csv --source-root ../../Materials/ADHD200/RawDataBIDS --dest-root SortedRawDataBIDS/cohort_adult_participants --id-column ScanDirID --site-column Site --sites-column Sites --sex-column Gender --age-column Age --diagnosis-column DX --mode symlink --dry-run

Код возврата: Done. See: /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/ADHD_connectome_biomarkers_transfer_research/ADHD200/SortedRawDataBIDS/cohort_adult_participants/flatten_report.tsv and /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/ADHD_connectome_biomarkers_transfer_research/ADHD200/SortedRawDataBIDS/cohort_adult_participants/participants.tsv

Ожидаемый отчёт: SortedRawDataBIDS/cohort_adult_participants/flatten_report.tsv


### 4.2 Предпросмотр отчёта DRY-RUN

In [90]:
print(f"\n--- 4.2.1 Предпросмотр отчёта DRY-RUN на возрастной когорте {TEEN_MINIMAL_AGE} - {TEEN_MAXIMAL_AGE} лет ---")

report = Path(DEST_ROOT_TEEN) / "flatten_report.tsv"
if report.exists():
    df = pd.read_csv(report, sep="\t")
    display(df.head(50))
    print("Всего строк в отчёте:", len(df))
    if "status" in df.columns:
        print("\nСтатистика по статусам:")
        print(df["status"].value_counts())
else:
    print(f"{report} не найден.")


--- 4.2.1 Предпросмотр отчёта DRY-RUN на возрастной когорте 12 - 15.99 лет ---


,orig_scandir_id,orig_site,sites_tried,status,src,dst,dst_subject,src_subject
0,2026113,3,KKI,would-link-anat,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN
1,2026113,3,KKI,would-link-func,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN
2,2026113,3,KKI,ok:copied=0;missing_sidecars=0,NaN,NaN,sub-2026113,/Users/alexey.stafeev/Documents/!Documents/Маг...
3,1623716,3,KKI,would-link-anat,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN
4,1623716,3,KKI,would-link-func,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN
5,1623716,3,KKI,ok:copied=0;missing_sidecars=0,NaN,NaN,sub-1623716,/Users/alexey.stafeev/Documents/!Documents/Маг...
6,1594156,3,KKI,would-link-anat,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN
7,1594156,3,KKI,would-link-func,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN
8,1594156,3,KKI,ok:copied=0;missing_sidecars=0,NaN,NaN,sub-1594156,/Users/alexey.stafeev/Documents/!Documents/Маг...
9,2917777,3,KKI,would-link-anat,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN


Всего строк в отчёте: 919

Статистика по статусам:
status
would-link-func                   371
would-link-anat                   274
ok:copied=0;missing_sidecars=0    274
Name: count, dtype: int64


In [91]:
print(f"\n--- 4.2.2 Предпросмотр отчёта DRY-RUN на возрастной когорте {ADULT_MINIMAL_AGE} - {ADULT_MAXIMAL_AGE} лет ---")

report = Path(DEST_ROOT_ADULT) / "flatten_report.tsv"
if report.exists():
    df = pd.read_csv(report, sep="\t")
    display(df.head(50))
    print("Всего строк в отчёте:", len(df))
    if "status" in df.columns:
        print("\nСтатистика по статусам:")
        print(df["status"].value_counts())
else:
    print(f"{report} не найден.")


--- 4.2.2 Предпросмотр отчёта DRY-RUN на возрастной когорте 16 - 21.99 лет ---


,orig_scandir_id,orig_site,sites_tried,status,src,dst,dst_subject,src_subject
0,1125505,4,NeuroIMAGE,would-link-anat,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN
1,1125505,4,NeuroIMAGE,would-link-func,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN
2,1125505,4,NeuroIMAGE,ok:copied=0;missing_sidecars=0,NaN,NaN,sub-1125505,/Users/alexey.stafeev/Documents/!Documents/Маг...
3,1208586,4,NeuroIMAGE,would-link-anat,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN
4,1208586,4,NeuroIMAGE,would-link-func,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN
5,1208586,4,NeuroIMAGE,ok:copied=0;missing_sidecars=0,NaN,NaN,sub-1208586,/Users/alexey.stafeev/Documents/!Documents/Маг...
6,1588809,4,NeuroIMAGE,would-link-anat,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN
7,1588809,4,NeuroIMAGE,would-link-func,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN
8,1588809,4,NeuroIMAGE,ok:copied=0;missing_sidecars=0,NaN,NaN,sub-1588809,/Users/alexey.stafeev/Documents/!Documents/Маг...
9,2029723,4,NeuroIMAGE,would-link-anat,/Users/alexey.stafeev/Documents/!Documents/Маг...,/Users/alexey.stafeev/Documents/!Documents/Маг...,NaN,NaN


Всего строк в отчёте: 387

Статистика по статусам:
status
would-link-func                   153
would-link-anat                   117
ok:copied=0;missing_sidecars=0    117
Name: count, dtype: int64


### 4.3 Полноценная конвертация (запись в DEST_ROOT_TEEN) для возрастной когорты {TEEN_MINIMAL_AGE} - {TEEN_MAXIMAL_AGE} лет

In [92]:
# Запускаем реальное копирование/создание симлинков для возрастной когорты {TEEN_MINIMAL_AGE} - {TEEN_MAXIMAL_AGE} лет
print(f"\n--- 4.3 Запуск реального копирования/создания симлинков для возрастной когорты {TEEN_MINIMAL_AGE} - {TEEN_MAXIMAL_AGE} лет ---")

ret = flatten_bids(PARTICIPANTS_CSV_TEEN, SOURCE_ROOT, DEST_ROOT_TEEN, ID_COLUMN, SITE_COLUMN, SITES_COLUMN, SEX_COLUMN, AGE_COLUMN, DIAGNOSIS_COLUMN, mode=MODE, dry_run=False)

print("\nКод возврата:", ret)

print(f" - participants.tsv в корне {DEST_ROOT_TEEN}")
print(f" - dataset_description.json в корне {DEST_ROOT_TEEN}")
print(" - sub-*/anat|func|fmap… с парными *.json и *.nii(.gz)")


--- 4.3 Запуск реального копирования/создания симлинков для возрастной когорты 12 - 15.99 лет ---
Выполняется команда: python flatten_bids_from_participants.py --participants-csv SortedRawDataBIDS/cohort_teen_participants/participants_teen.csv --source-root ../../Materials/ADHD200/RawDataBIDS --dest-root SortedRawDataBIDS/cohort_teen_participants --id-column ScanDirID --site-column Site --sites-column Sites --sex-column Gender --age-column Age --diagnosis-column DX --mode symlink

Код возврата: Done. See: /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/ADHD_connectome_biomarkers_transfer_research/ADHD200/SortedRawDataBIDS/cohort_teen_participants/flatten_report.tsv and /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/ADHD_connectome_biomarkers_transfer_research/ADHD200/SortedRawDataBIDS/cohort_teen_participants/participants.tsv

 - participants.tsv в корне SortedRawDataBIDS/cohort_teen_participants
 - dataset_description.json в корне 

### 4.4 Полноценная конвертация (запись в DEST_ROOT_ADULT) для возрастной когорты {ADULT_MINIMAL_AGE} - {ADULT_MAXIMAL_AGE} лет

In [93]:
# Запускаем реальное копирование/создание симлинков для возрастной когорты {ADULT_MINIMAL_AGE} - {ADULT_MAXIMAL_AGE} год
print(f"\n--- 4.4 Запуск реального копирования/создания симлинков для возрастной когорты {ADULT_MINIMAL_AGE} - {ADULT_MAXIMAL_AGE} год ---")

ret = flatten_bids(PARTICIPANTS_CSV_ADULT, SOURCE_ROOT, DEST_ROOT_ADULT, ID_COLUMN, SITE_COLUMN, SITES_COLUMN, SEX_COLUMN, AGE_COLUMN, DIAGNOSIS_COLUMN, mode=MODE, dry_run=False)

print("\nКод возврата:", ret)

print(f" - participants.tsv в корне {DEST_ROOT_ADULT}")
print(f" - dataset_description.json в корне {DEST_ROOT_ADULT}")
print(" - sub-*/anat|func|fmap… с парными *.json и *.nii(.gz)")


--- 4.4 Запуск реального копирования/создания симлинков для возрастной когорты 16 - 21.99 год ---
Выполняется команда: python flatten_bids_from_participants.py --participants-csv SortedRawDataBIDS/cohort_adult_participants/participants_adult.csv --source-root ../../Materials/ADHD200/RawDataBIDS --dest-root SortedRawDataBIDS/cohort_adult_participants --id-column ScanDirID --site-column Site --sites-column Sites --sex-column Gender --age-column Age --diagnosis-column DX --mode symlink

Код возврата: Done. See: /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/ADHD_connectome_biomarkers_transfer_research/ADHD200/SortedRawDataBIDS/cohort_adult_participants/flatten_report.tsv and /Users/alexey.stafeev/Documents/!Documents/Магистратура_МФТИ/Предметы/НИР/ADHD_connectome_biomarkers_transfer_research/ADHD200/SortedRawDataBIDS/cohort_adult_participants/participants.tsv

 - participants.tsv в корне SortedRawDataBIDS/cohort_adult_participants
 - dataset_description.json в 